#Работа с pickle

Попробуем сериализовать класс путем записи его в файл.

За это отвечает команда **pickle.dump(obj, file)**

In [ ]:
import pickle

class User:
  def __init__(self, age, name):
    self.age = age
    self.name = name
  def get_values(self):
    print(f"{self.age=}")
    print(f"{self.name=}")

with open("data.pickle", "wb") as f:
  pickle.dump(User, f)

всё записалось в файл!
Теперь можно его передать на другой ПК.

А теперь попробуем получить обратно этот класс из файла.

In [ ]:
with open('data.pickle', 'rb') as f:
  user = pickle.load(f)

In [ ]:
print(user)

<class '__main__.User'>


Действительно получили :)

То есть мы можем брать наш старый сериализованный код, включать его в любую нашу новую программу и использовать заново.

In [ ]:
a = user(10, "test")
a.get_values()

self.age=10
self.name='test'


а что, если мы хотим использовать сериализацию внутри одной программы, внутри одного ПК и нам не надо создавать файл?

ну и не будем создавать тогда.

методы **dumps** и **loads** позволяют это!

In [ ]:
import pickle

a = {1:2, 3: [1, 2, 3]}
b = pickle.dumps(a)
print(b)

b'\x80\x04\x95\x15\x00\x00\x00\x00\x00\x00\x00}\x94(K\x01K\x02K\x03]\x94(K\x01K\x02K\x03eu.'


**loads** -- загружает сериализованные данные

In [ ]:
 d = pickle.loads(b)
 print(d)

{1: 2, 3: [1, 2, 3]}



**Опасность использования pickle:**

мы можем исполнить код злоумышленников. Мы же не знаем, что сериализовано. Так что не рекомендуется использовать pickle в программах, которые получают на вход какие-то данные, которые вводят сторонние пользователи (заполняют формы, анкеты)

**пример:**


In [ ]:
class Attack:
  def __reduce__(self):
    return (exec, ("import webbrowser; webbrowser.open('https://duckduckgo.com')",))

malicious = pickle.dumps(Attack())
pickle.loads(malicious)


**что делать, если надо всё-таки сериализовать данные пользователей?**

Просто возьмите и используйте json вместо pickle.

Работает точно так же. Только не принимает на десериализацию команды консольные, переходы на сайты и всякое такое.

In [ ]:
import json


a = json.dumps([1, 2, 3, {'4': 5, '6': 7}], separators=(',', ':'))
print(a)

[1,2,3,{"4":5,"6":7}]


//dump pickle, json load -- нельзя, потому что форматы разные!

**Вывод:**

1. dump -- записываем внутрь файла

dumps -- внутрь переменной

2. json вместо pickle, если думаете, что вам на десериализацию могут попасться опасные данные

##Обработка объектов с учётом состояния

Класс TextReader открывает текстовый файл и возвращает номер строки и содержимое строки при каждом вызове метода readline(). Если TextReader сущность является pickled, все атрибуты кроме элемента объекта файла сохраняются. Когда сущность - unpickled, файл вновь открыт, и читающий возобновляет от последнего местоположения. Методы __setstate__() и __getstate__() являются используемый для реализации этого поведения:

In [ ]:
class TextReader:
    """Печать и нумерация строк в текстовом файле."""

    def __init__(self, filename):
        self.filename = filename
        self.file = open(filename)
        self.lineno = 0

    def readline(self):
        self.lineno += 1
        line = self.file.readline()
        if not line:
            return None
        if line.endswith('\n'):
            line = line[:-1]
        return "%i: %s" % (self.lineno, line)

    def __getstate__(self):
        # Скопировать объект состояние из self.__dict__, который содержит все нашей сущность
        # атрибуты. Всегда используйте dict.copy() метод, чтобы не изменять
        # оригинальный состояние.
        state = self.__dict__.copy()
        # Удалите недопустимые (unpicklable) записи.
        del state['file']
        return state

    def __setstate__(self, state):
        # Восстановить атрибуты сущности (то есть имя файла и lineno).
        self.__dict__.update(state)
        # Восстановите состояние ранее открытого файла. Для этого нам нужно снова открыть
        # его и читать с него до тех пор, пока не будет восстановлен счетчик строк.
        file = open(self.filename)
        for _ in range(self.lineno):
            file.readline()
        # Наконец, сохранить файл.
        self.file = file

In [ ]:
file = open("hello.txt", "w+")
file.write("Hello! \n Have a good day! \n Goodbye!")
file.close()

In [ ]:
reader = TextReader("hello.txt")
reader.readline()

'1: Hello! '

In [ ]:
reader.readline()

'2:  Have a good day! '

In [ ]:
new_reader = pickle.loads(pickle.dumps(reader))

In [ ]:
new_reader.readline()

**вывод**


1. JSON — это формат сериализации текста (он выводит текст в формате Юникод, хотя большую часть времени он закодирован в utf-8), а pickle — это двоичный формат сериализации;
2. JSON удобочитаемый, а pickle — нет;
3. JSON совместим и широко используется за пределами экосистемы Python, в то время как pickle специфичен для Python;
4. JSON по умолчанию может представлять только подмножество встроенных типов Python и не может представлять собой пользовательские классы; pickle может представлять чрезвычайно большое количество типов Python (многие из них автоматически, с умным использованием средств самоанализа Python; сложные случаи могут быть решены путём реализации API объектов) ;
5. В отличие от pickle, десериализация ненадежного JSON сама по себе не создаёт уязвимости при выполнении произвольного кода.

**Полезные ссылки:**

1. [17 минут объяснение всего пикл на английском (тут про объекты с состоянием есть)](https://www.youtube.com/watch?v=6Q56r_fVqgw&ab_channel=Indently)